<a href="https://colab.research.google.com/github/edmarja/projects-datascience/blob/main/case_aerolinea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Escenario:

Estás analizando la "evolución temporal" del movimiento humano (una manera elegante de decir cómo cambia el movimiento humano con el tiempo) en un aeropuerto específico. Y decides que necesitas obtener recuentos diarios de pasajeros que llegan para cada aerolínea durante muchos años.
Encontrarás dos conjuntos de datos que contienen esta información:

* Un conjunto de datos público no afiliado al aeropuerto ni a las aerolíneas que abarca 10 años.
* Un conjunto de datos del aeropuerto que abarca seis meses.

De buenas a primeras es evidente que el conjunto de datos del aeropuerto es más fiable. Sin embargo, seis meses es un período de tiempo demasiado corto para tu análisis. Te encantaría utilizar el conjunto de datos de 10 años, pero necesitas saber si es fiable. Para tener una idea de la precisión de los datos públicos, compararás los recuentos dentro del mismo período de seis meses entre los dos conjuntos de datos. Aplicarás formato a ambos conjuntos de datos para mantener la coherencia, combinarás los datos y utilizarás tablas dinámicas para examinar rápidamente las diferencias entre los recuentos. 

Esperando que la diferencia en los recuentos sea cercana a 0, lo que implicaría que estos datos públicos son precisos. Aquí vamos...

In [ ]:
import pandas as pd

In [ ]:
airportArrivals = pd.read_excel('/content/PassengerTotals.xlsx', sheet_name='Source1')

print(airportArrivals.info())
airportArrivals.tail(10)

#Estudia el resultado. ¿Cuáles son los tipos de datos de la variable? ¿Cuáles deberían ser?
#También observa que nuestro DataFrame tiene 905 filas y tenemos 905 valores no nulos en cada columna. 
# Eso significa que no tenemos que preocuparnos por valores ausentes en nuestro análisis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Airlines    905 non-null    object
 1   Date        905 non-null    object
 2   Passengers  905 non-null    object
dtypes: object(3)
memory usage: 21.3+ KB
None


,Airlines,Date,Passengers
895,Alaska,29/06/19 01:00:00,03690
896,Delta,29/06/19 01:00:00,01993
897,Frontier,29/06/19 01:00:00,02440
898,JetBlue,29/06/19 01:00:00,01800
899,Southwest,29/06/19 01:00:00,03862
900,Alaska,30/06/19 01:00:00,03515
901,Delta,30/06/19 01:00:00,01915
902,Frontier,30/06/19 01:00:00,02532
903,JetBlue,30/06/19 01:00:00,01873
904,Southwest,30/06/19 01:00:00,03568


In [ ]:
# Intentamos cambiar 'Passengers' a tipo numérico
try:
    airportArrivals['Passengers'] = pd.to_numeric(airportArrivals['Passengers'])
except:
    print('Error: conversión numérica. Por favor comprobar entradas en la columna Passenger')

Error: conversión numérica. Por favor comprobar entradas en la columna Passenger


In [ ]:
#Tenemos que transformar la columna 'Passengers' al tipo de datos numéricos para poder comparar los 
# dos conjuntos de datos. Por eso necesitamos encontrar los valores en 'Passengers' que no pudimos convertir. Vamos a hacerlo ahora.

problem_entries = []


for value in airportArrivals['Passengers']:
    try:
        pd.to_numeric(value)
    except:
        problem_entries.append(value)

problem_entries

#​Todo este trabajo (necesario) era para encontrar una entrada que tiene un apóstrofo después del último dígito.

["01897'"]

In [ ]:
#Como solo hay un error trabajemoslo manualmente

airportArrivals.loc[airportArrivals['Passengers'] == "01897'"] #ubicamos en que fila se ubica la cadena 01897'

airportArrivals.loc[32,'Passengers'] = airportArrivals.loc[32,'Passengers'].replace("01897'", '01897') #reemplazamos el valor por '01897'

#airportArrivals.loc[airportArrivals['Passengers'] == "01897'", 'Passengers'] = '01897' otra forma de reemplazar por '01897'

airportArrivals['Passengers'] = pd.to_numeric(airportArrivals['Passengers']) #convertimos la columna 'Passengers' a número

airportArrivals.info() # ahora la columna 'Passenger' son enteros

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Airlines    905 non-null    object
 1   Date        905 non-null    object
 2   Passengers  905 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.3+ KB


In [ ]:
#ahora tenemos que convertir la columna Date a formato datetime

dateFormat = '%d/%m/%y %H:%M:%S'
airportArrivals['Date'] = pd.to_datetime(airportArrivals['Date'], format= dateFormat) #convertimos a datetime

airportArrivals.info()
airportArrivals.head()
#Ahora las columnas 'Passengers' y 'Date' son los tipos de datos correctos, así que podemos realizar nuestro análisis.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Airlines    905 non-null    object        
 1   Date        905 non-null    datetime64[ns]
 2   Passengers  905 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 21.3+ KB


,Airlines,Date,Passengers
0,Alaska,2019-01-01 01:00:00,2672
1,Delta,2019-01-01 01:00:00,1491
2,Frontier,2019-01-01 01:00:00,1965
3,JetBlue,2019-01-01 01:00:00,1496
4,Southwest,2019-01-01 01:00:00,2922


La hoja de Excel 'Source2' contiene datos del conjunto de datos públicos que cubre 10 años, incluido el período de seis meses en los datos del aeropuerto. Carguemos estos datos y veamos qué formateo debemos hacer.

Lee la hoja 'Source2' del mismo archivo de Excel. Muestra las primeras 10 líneas e imprime los tipos de datos.

In [ ]:
publicArrivals = pd.read_excel('PassengerTotals.xlsx', sheet_name='Source2')

print(publicArrivals.info())
publicArrivals.head(10)

#Parece que los datos de 'Passengers' se cargaron correctamente como números enteros. 
# Todo lo que tenemos que hacer es convertir 'Date' a datetime y 'Passengers' a int16.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Airlines    905 non-null    object
 1   Date        905 non-null    object
 2   Passengers  905 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.3+ KB
None


,Airlines,Date,Passengers
0,Alaska,01/01/19 01:00:00,3044
1,Delta,01/01/19 01:00:00,1529
2,Frontier,01/01/19 01:00:00,1665
3,JetBlue,01/01/19 01:00:00,1648
4,Southwest,01/01/19 01:00:00,3167
5,Alaska,02/01/19 01:00:00,2846
6,Delta,02/01/19 01:00:00,1298
7,Frontier,02/01/19 01:00:00,1648
8,JetBlue,02/01/19 01:00:00,1570
9,Southwest,02/01/19 01:00:00,3231


In [ ]:
#Queremos que nuestros conjuntos de datos tengan un formato coherente para poder fusionar y comparar valores

dateFormat = '%d/%m/%y %H:%M:%S'

publicArrivals['Date'] = pd.to_datetime(publicArrivals['Date'], format= dateFormat) #cambiamos a formato datetime

publicArrivals['Passengers'] = publicArrivals['Passengers'].astype('int16') #pasamos a entero int16

publicArrivals.info()

publicArrivals.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Airlines    905 non-null    object        
 1   Date        905 non-null    datetime64[ns]
 2   Passengers  905 non-null    int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 16.0+ KB


,Airlines,Date,Passengers
0,Alaska,2019-01-01 01:00:00,3044
1,Delta,2019-01-01 01:00:00,1529
2,Frontier,2019-01-01 01:00:00,1665
3,JetBlue,2019-01-01 01:00:00,1648
4,Southwest,2019-01-01 01:00:00,3167


Ahora que tenemos un formato coherente, es el momento de combinar los conjuntos de datos. Así todos nuestros datos estarán en una tabla y eso facilitará mucho el trabajo con los datos.

In [ ]:
passenger_compare = airportArrivals.merge(right=publicArrivals, on=['Airlines', 'Date'], how='left')

passenger_compare.head(10)

#Dado que ambos conjuntos de datos tenían la columna 'Passengers', Python añadió automáticamente _x (al DataFrame izquierdo) 
# y _y (al DataFrame derecho) para distinguir entre las dos columnas únicas.

,Airlines,Date,Passengers_x,Passengers_y
0,Alaska,2019-01-01 01:00:00,2672,3044
1,Delta,2019-01-01 01:00:00,1491,1529
2,Frontier,2019-01-01 01:00:00,1965,1665
3,JetBlue,2019-01-01 01:00:00,1496,1648
4,Southwest,2019-01-01 01:00:00,2922,3167
5,Alaska,2019-01-02 01:00:00,2717,2846
6,Delta,2019-01-02 01:00:00,1524,1298
7,Frontier,2019-01-02 01:00:00,1896,1648
8,JetBlue,2019-01-02 01:00:00,1500,1570
9,Southwest,2019-01-02 01:00:00,2762,3231


### **Realización de análisis con la tabla fusionada**

Pensemos en cómo queremos comparar los datos de 'Passengers' de los dos conjuntos de datos. Nuestros datos solo abarcan seis meses, por lo que no podemos hacer ninguna comparación por año; sin embargo, podemos ver la diferencia media entre los recuentos de 'Passengers' por mes y día de la semana. Vamos a crear algunas variables nuevas que lo harán posible.

In [ ]:
passenger_compare['Month'] = pd.DatetimeIndex(passenger_compare['Date']).month
passenger_compare['dayOfWeek'] = pd.DatetimeIndex(passenger_compare['Date']).dayofweek
passenger_compare['difference'] = passenger_compare['Passengers_y'] - passenger_compare['Passengers_x']

passenger_compare.head(10)

#Ten en cuenta que el atributo dayofweek devuelve valores entre 0 y 6, donde 0 = lunes y 6 = domingo.

,Airlines,Date,Passengers_x,Passengers_y,Month,dayOfWeek,difference
0,Alaska,2019-01-01 01:00:00,2672,3044,1,1,372
1,Delta,2019-01-01 01:00:00,1491,1529,1,1,38
2,Frontier,2019-01-01 01:00:00,1965,1665,1,1,-300
3,JetBlue,2019-01-01 01:00:00,1496,1648,1,1,152
4,Southwest,2019-01-01 01:00:00,2922,3167,1,1,245
5,Alaska,2019-01-02 01:00:00,2717,2846,1,2,129
6,Delta,2019-01-02 01:00:00,1524,1298,1,2,-226
7,Frontier,2019-01-02 01:00:00,1896,1648,1,2,-248
8,JetBlue,2019-01-02 01:00:00,1500,1570,1,2,70
9,Southwest,2019-01-02 01:00:00,2762,3231,1,2,469


In [ ]:
#Comparemos los recuentos de los dos conjuntos de datos resumiendo la columna 'difference'. 
# Primero, veamos la media de nuestra columna 'difference' por 'Airlines' y 'Month'.

pivot_table_month = passenger_compare.pivot_table(index= 'Airlines', columns='Month', values='difference', aggfunc='mean')

pivot_table_month.head()

Month,1,2,3,4,5,6
Airlines,,,,,,
Alaska,-269.935484,-302.892857,-336.838710,-230.433333,-216.419355,-337.233333
Delta,-233.903226,-261.857143,-279.580645,-252.100000,-240.000000,-247.733333
Frontier,-324.129032,-330.821429,-348.709677,-265.366667,-282.935484,-351.600000
JetBlue,-11.709677,-11.071429,-27.580645,16.100000,-16.000000,-43.066667
Southwest,27.387097,-24.785714,-80.258065,19.466667,105.483871,-128.166667


In [ ]:
# Ahora hacemos la comparación por días de la semana

pivot_table_day = passenger_compare.pivot_table(index='Airlines', columns='dayOfWeek', values='difference', aggfunc='mean')

pivot_table_day.head()

dayOfWeek,0,1,2,3,4,5,6
Airlines,,,,,,,
Alaska,91.32,17.923077,-10.192308,-80.730769,58.153846,-1021.846154,-1013.807692
Delta,-153.20,-150.692308,-199.384615,-186.115385,-172.307692,-453.000000,-448.307692
Frontier,-183.40,-186.961538,-196.307692,-166.230769,-204.730769,-658.730769,-618.423077
JetBlue,139.04,141.000000,148.115385,136.461538,170.346154,-427.269231,-411.307692
Southwest,288.20,336.769231,372.269231,325.384615,204.038462,-771.307692,-833.653846


## CONCLUSION

Aquí está nuestra interpretación de los resultados...

De pivot_table_month, parece que los recuentos más precisos pertenecen a JetBlue, seguido de Southwest. Para estas dos aerolíneas, el conjunto de datos público sobreestima (diferencia positiva) y subestima (diferencia negativa) los recuentos de llegadas a lo largo de los meses. Para las otras tres aerolíneas (Alaska, Delta y Frontier), el conjunto de datos público subestima sistemáticamente los recuentos de llegadas a lo largo de los meses.

Se puede extraer muchos conocimientos nuevos de pivot_table_day. Resulta que los recuentos de las aerolíneas de Alaska son mucho mejores de lunes a viernes, pero se subestiman gravemente para los sábados y domingos. Delta y Frontier son constantemente subestimadas a lo largo de la semana. Los recuentos de JetBlue y Southwest parecían ser más precisos en nuestros datos mensuales, sin embargo, los datos diarios revelan que ambas aerolíneas en realidad están sobreestimadas los días de semana y subestimadas los fines de semana. Cuando comparamos los días de semana con los fines de semana, vemos que, para cada aerolínea, los recuentos de los días de semana son generalmente más precisos (más cercanos a 0) en comparación con los recuentos de los fines de semana.

Resulta que hay algunos problemas en el conjunto de datos público. Sin embargo, puede ser difícil encontrar los datos que necesitas. Si decides utilizar el conjunto de datos público, debes comprender dónde están los problemas y en qué medida podrían afectar a tu análisis.